# Pre-processing and cleaning approved drugs (ChEMBL Web Resource)

In this step, we drop some missing and irrelevant stuff and reorder stuff. The objective of this notebook is to do the pre-cleaning and cleaning of the dataset gathered from ChEMBL Web Resource. More info about it below.

The ChEMBL Web Resource, as defined by the authors and mantainers in [GitHub](https://github.com/chembl/chembl_webresource_client): *'The library helps accessing ChEMBL data and cheminformatics tools from Python. You don't need to know how to write SQL. You don't need to know how to interact with REST APIs. You don't need to compile or install any cheminformatics frameworks. Results are cached.'*

## Importing packages and data

This section is dedicated to **import the packages and libraries** that we're going to use in this notebook, aswell as **the data** retrieved from *ChEMBL Web Resource* (more info above).

In [87]:
# Importing libraries
from rdkit import Chem, rdBase
from rdkit.Chem import Draw, Descriptors, PandasTools, AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.SaltRemover import SaltRemover, InputFormat
import pandas as pd
import re

In [88]:
# Importing the dataset:
approved_drugs_df = pd.read_csv('data/RAW_approved_drugs.csv')

## Step 1 - Pre-process

Taking a peek into the data we can see that there are a lot of variables that we don't need aswell as some rows depicting "NaN". We need to filter the bulk of information then refine it. This is the objective of the pre-process step.

In [89]:
# List of variables
print(approved_drugs_df.columns, "\n\n",approved_drugs_df.shape )

# Taking a peek into the data
approved_drugs_df.head(5)

Index(['atc_classifications', 'availability_type', 'biotherapeutic',
       'black_box_warning', 'chebi_par_id', 'chirality', 'cross_references',
       'dosed_ingredient', 'first_approval', 'first_in_class', 'helm_notation',
       'indication_class', 'inorganic_flag', 'max_phase', 'molecule_chembl_id',
       'molecule_hierarchy', 'molecule_properties', 'molecule_structures',
       'molecule_synonyms', 'molecule_type', 'natural_product', 'oral',
       'parenteral', 'polymer_flag', 'pref_name', 'prodrug', 'structure_type',
       'therapeutic_flag', 'topical', 'usan_stem', 'usan_stem_definition',
       'usan_substem', 'usan_year', 'withdrawn_class', 'withdrawn_country',
       'withdrawn_flag', 'withdrawn_reason', 'withdrawn_year', 'SMILES'],
      dtype='object') 

 (4121, 39)


,atc_classifications,availability_type,biotherapeutic,black_box_warning,chebi_par_id,chirality,cross_references,dosed_ingredient,first_approval,first_in_class,...,usan_stem,usan_stem_definition,usan_substem,usan_year,withdrawn_class,withdrawn_country,withdrawn_flag,withdrawn_reason,withdrawn_year,SMILES
0,['V03AN03'],1,NaN,0,30217.0,2,[],True,2015.0,0,...,-ium,quaternary ammonium derivatives,-ium,NaN,NaN,NaN,False,NaN,NaN,[He]
1,[],1,NaN,0,16134.0,2,"[{'xref_id': 'ammonia%20n-13', 'xref_name': 'a...",False,2007.0,0,...,NaN,NaN,NaN,1990.0,NaN,NaN,False,NaN,NaN,N
2,[],1,NaN,0,NaN,2,"[{'xref_id': 'ammonia%20n-13', 'xref_name': 'a...",True,2007.0,0,...,NaN,NaN,NaN,1990.0,NaN,NaN,False,NaN,NaN,[13NH3]
3,[],2,NaN,0,15377.0,2,"[{'xref_id': 'purified%20water', 'xref_name': ...",True,2011.0,0,...,deu-,deuterated compounds,deu-,1963.0,NaN,NaN,False,NaN,NaN,O
4,['V03AN04'],1,NaN,0,17997.0,2,[],True,2015.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,N#N


We can filter the information of only those variables:
* pref_name
* SMILES
* first_approval
* molecule_type
* indication_class
* polymer_flag
* withdrawn_flag
* inorganic_flag
* therapeutic_flag
* natural_product
* oral
* parenteral
* topical

In [90]:
approved_drugs_df = approved_drugs_df[[
    'pref_name',
    'SMILES',
    'first_approval',
    'molecule_type',
    'indication_class',
    'polymer_flag',
    'inorganic_flag',
    'withdrawn_flag',
    'therapeutic_flag',
    'natural_product',
    'oral', 
    'parenteral', 
    'topical',
]]

In [91]:
# Taking a look into approved drugs
approved_drugs_df

,pref_name,SMILES,first_approval,molecule_type,indication_class,polymer_flag,inorganic_flag,withdrawn_flag,therapeutic_flag,natural_product,oral,parenteral,topical
0,HELIUM,[He],2015.0,Small molecule,"Gases, Diluent for",0,1,False,False,0,False,False,True
1,"AMMONIA SOLUTION, STRONG",N,2007.0,Small molecule,Pharmaceutic Aid (solvent and source of ammoni...,0,0,False,False,0,False,True,False
2,AMMONIA N 13,[13NH3],2007.0,Small molecule,Radioactive Agent; Diagnostic Aid (cardiac ima...,0,0,False,False,0,False,True,False
3,WATER,O,2011.0,Small molecule,"Diagnostic Aid (radioactive, vascular disorder...",0,0,False,False,0,False,False,True
4,NITROGEN,N#N,2015.0,Small molecule,Pharmaceutic Aid (air displacement),0,0,False,False,0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,DINUTUXIMAB,not found,2015.0,Antibody,NaN,0,0,False,True,0,False,True,False
4117,PEMBROLIZUMAB,not found,2014.0,Antibody,NaN,0,0,False,True,0,False,True,False
4118,TRASTUZUMAB DERUXTECAN,not found,2019.0,Antibody,NaN,0,0,False,True,0,False,True,False
4119,DASIGLUCAGON HYDROCHLORIDE,not found,2021.0,Unknown,NaN,0,0,False,True,0,False,True,False


Next we do the following pre-cleaning steps:
* Filter only the *Small molecules* on the `molecule_type` variable;
* Remove strictly *inorganic* and *polymers*;
* Remove SMILES not found;
* Remove radioactive, gases and diluent;
* Only `therapeutic_flag` == True;
* Remove structures without approval date;

In [92]:
# Filtering the structures:

# We want only 'Small molecule', don't want inorganic or polymer stuff
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['molecule_type'] == 'Small molecule']
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['inorganic_flag'] == 0]
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['polymer_flag'] == 0]

# We don't want SMILES not found
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['SMILES'] != 'not found']

# We don't want indication_class containing the word "gases" or "diluent" or "radioactive"
approved_drugs_df = approved_drugs_df[~approved_drugs_df['indication_class'].str.contains("radioactive|gases|diluent", flags=re.IGNORECASE, regex=True, na=False)]

# with terapeutic_flag == True
approved_drugs_df = approved_drugs_df[approved_drugs_df['therapeutic_flag'] == True]

# Remove the undated first approvals
approved_drugs_df = approved_drugs_df[~approved_drugs_df['first_approval'].isna()].reset_index(drop=True)

In [93]:
approved_drugs_df.head(10)

,pref_name,SMILES,first_approval,molecule_type,indication_class,polymer_flag,inorganic_flag,withdrawn_flag,therapeutic_flag,natural_product,oral,parenteral,topical
0,NITRIC OXIDE,[N]=O,1999.0,Small molecule,NaN,0,0,False,True,0,False,False,True
1,HYDROGEN PEROXIDE,OO,2017.0,Small molecule,"Anti-Infective, Topical",0,0,False,True,0,False,False,True
2,GUANIDINE,N=C(N)N,1939.0,Small molecule,NaN,0,0,False,True,0,True,False,False
3,GUANIDINE HYDROCHLORIDE,Cl.N=C(N)N,1939.0,Small molecule,NaN,0,0,False,True,0,True,False,False
4,LITHIUM CARBONATE,O=C([O-])[O-].[Li+].[Li+],1970.0,Small molecule,Antimanic,0,0,False,True,0,True,False,False
5,ACETOHYDROXAMIC ACID,CC(=O)NO,1983.0,Small molecule,Enzyme Inhibitor (urease),0,0,False,True,0,True,False,False
6,HYDROXYUREA,NC(=O)NO,1967.0,Small molecule,Antineoplastic,0,0,False,True,0,True,False,False
7,CYSTEAMINE BITARTRATE,NCCS.O=C(O)C(O)C(O)C(=O)O,1994.0,Small molecule,NaN,0,0,False,True,0,True,False,False
8,CYSTEAMINE HYDROCHLORIDE,Cl.NCCS,2012.0,Small molecule,Anti-Urolithic (cystine calculi),0,0,False,True,0,False,False,True
9,CYSTEAMINE,NCCS,1994.0,Small molecule,Anti-Urolithic (cystine calculi),0,0,False,True,0,True,False,True


Now we can do some tweaking into our dataframe:
* **Rounding** the number of approval date (just for aesthetic)
* **Reordering** the dataframe into a more logic variable sequence

In [94]:
#Rounding the datetime
approved_drugs_df['first_approval'] = approved_drugs_df['first_approval'].apply(int)

# Reordering the dataframe
approved_drugs_df = approved_drugs_df[['pref_name','SMILES','first_approval','polymer_flag','indication_class','withdrawn_flag','inorganic_flag','polymer_flag','therapeutic_flag','natural_product','oral', 'parenteral', 'topical']]

print("\n", approved_drugs_df.columns, "\n")
print(f"The dataset has the shape {approved_drugs_df.shape}")
approved_drugs_df


 Index(['pref_name', 'SMILES', 'first_approval', 'polymer_flag',
       'indication_class', 'withdrawn_flag', 'inorganic_flag', 'polymer_flag',
       'therapeutic_flag', 'natural_product', 'oral', 'parenteral', 'topical'],
      dtype='object') 

The dataset has the shape (2396, 13)


,pref_name,SMILES,first_approval,polymer_flag,indication_class,withdrawn_flag,inorganic_flag,polymer_flag,therapeutic_flag,natural_product,oral,parenteral,topical
0,NITRIC OXIDE,[N]=O,1999,0,NaN,False,0,0,True,0,False,False,True
1,HYDROGEN PEROXIDE,OO,2017,0,"Anti-Infective, Topical",False,0,0,True,0,False,False,True
2,GUANIDINE,N=C(N)N,1939,0,NaN,False,0,0,True,0,True,False,False
3,GUANIDINE HYDROCHLORIDE,Cl.N=C(N)N,1939,0,NaN,False,0,0,True,0,True,False,False
4,LITHIUM CARBONATE,O=C([O-])[O-].[Li+].[Li+],1970,0,Antimanic,False,0,0,True,0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,TELAVANCIN,CCCCCCCCCCNCCN[C@@]1(C)C[C@H](O[C@H]2[C@H](Oc3...,2009,0,NaN,False,0,0,True,1,False,True,False
2392,TELAVANCIN HYDROCHLORIDE,CCCCCCCCCCNCCN[C@@]1(C)C[C@H](O[C@H]2[C@H](Oc3...,2009,0,NaN,False,0,0,True,1,False,True,False
2393,ERGOLOID,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]2(O)O[C@](NC(=...,1953,0,Cognition Adjuvant,False,0,0,True,1,True,False,True
2394,ERGOLOID MESYLATES,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]2(O)O[C@](NC(=...,1953,0,Cognition Adjuvant,False,0,0,True,1,True,False,True


Now we're left with the following problems:
* Extremely small structures such as Nitric Oxide, Hydrogen Peroxide that cannot be used to derive any rules. (Maybe this can be solved by applying some MW filter)
* Repeated SMILES (salt and conjugated) - Maybe this can be solved by keeping the largest fragment and dropping x*plicated SMILES.
* Extremely large structures such as OMEGA-3-Carboxylic Acids (Apply some MW filter? Maybe)

Solving the first problem (extremely small structures). We calculate the mol_format 